In [1]:
#Run cell to mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# install package to have access to custom functions
%pip install /content/drive/MyDrive/EMG_gestures/ --use-feature=in-tree-build

Processing ./drive/MyDrive/EMG_gestures
  Created wheel for EMG-gestures: filename=EMG_gestures-0.1.0-py3-none-any.whl size=37820 sha256=a568a25d8ce149ec11f59d0fac9f2668388d4043a4e61a5a7be94991ceb5e08a
  Stored in directory: /tmp/pip-ephem-wheel-cache-phjwl3fs/wheels/a2/b7/61/2147fa082a9e51bef5dcc38dd3f0898fe0554d62203c0e383e
Successfully built EMG-gestures
  Attempting uninstall: EMG-gestures
    Found existing installation: EMG-gestures 0.1.0
    Uninstalling EMG-gestures-0.1.0:
      Successfully uninstalled EMG-gestures-0.1.0


In [3]:
#import necessary packages

#our workhorses
import numpy as np
import pandas as pd
import scipy

#to visualize
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#style params for figures
sns.set(font_scale = 2)
plt.style.use('seaborn-white')
plt.rc("axes", labelweight="bold")
from IPython.display import display, HTML

#to load files
import os
import sys
import h5py

#import cusotm functions
from EMG_gestures.utils import *
from EMG_gestures.analysis import get_trained_rnn_model, evaluate_trained_rnn


In [15]:
#define where the data files are located
data_folder = '/content/drive/MyDrive/EMG_gestures/EMG_data/'
results_folder = '/content/drive/MyDrive/EMG_gestures/results_data/single_subject_training/RNN/'

nsubjects = 5


# User-defined parameters
lo_freq = 20 #lower bound of bandpass filter
hi_freq = 450 #upper bound of bandpass filter

win_size = 100 #define window size over which to compute time-domain features
step = win_size #keeping this parameter in case we want to re-run later with some overlap


#for RNN training
epochs = 40
batch_size = 2


nreps = 10
nsets_training = 10
exclude = [0,7]#labels to exclude
#performance metrics
score_list = ['f1','accuracy']

src_subject_id = 1
#for src_subject_id in range(1,nsubjects+1):

results_df = rnn_xsubject_test(data_folder, src_subject_id, nsubjects, nreps, lo_freq, hi_freq, win_size, step, exclude, score_list,\
                        nsets_training, epochs = epochs, batch_size = batch_size, mv = None, permute = False)

results_df['Shuffled'] = False


/content/drive/MyDrive/EMG_gestures/EMG_data/01
Subject 1|Rep 1
Training Model
Evaluate Model on Trained Data
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/02
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/03
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/04
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/05
Evaluate Model
Subject 1|Rep 2
Training Model
Evaluate Model on Trained Data
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/02
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/03
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/04
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/05
Evaluate Model
Subject 1|Rep 3
Training Model
Evaluate Model on Trained Data
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/02
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG

NameError: ignored

In [16]:

perm_results_df = rnn_xsubject_test(data_folder, src_subject_id, nsubjects, nreps, lo_freq, hi_freq, win_size, step, exclude, score_list,\
                        nsets_training, epochs = epochs, batch_size = batch_size, mv =None, permute = True)

perm_results_df['Shuffled'] = True

#concatenate

results_df = pd.concat([results_df, perm_results_df]).reset_index(drop = True)
results_df['Train_Subject'] = src_subject_id

# # #save results to file
# results_fn = 'subject_%02d_across_subject_results.h5'%(src_subject_id)
# results_df.to_hdf(os.path.join(results_folder,results_fn), key='results_df', mode='w')

/content/drive/MyDrive/EMG_gestures/EMG_data/01
Subject 1|Rep 1
Training Model
Evaluate Model on Trained Data
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/02
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/03
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/04
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/05
Evaluate Model
Subject 1|Rep 2
Training Model
Evaluate Model on Trained Data
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/02
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/03
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/04
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/05
Evaluate Model
Subject 1|Rep 3
Training Model
Evaluate Model on Trained Data
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/02
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG

In [17]:
results_df.groupby(['Shuffled','Type']).mean()

Rep  Test_Subject  f1_score  accuracy_score  Train_Subject
Shuffled Type                                                             
False    Test   5.5           3.5  0.582665        0.626036            1.0
         Train  5.5           1.0  0.960589        0.960584            1.0
True     Test   5.5           3.5  0.130660        0.158955            1.0
         Train  5.5           1.0  0.683433        0.688995            1.0

In [14]:
def rnn_xsubject_test(data_folder, src_subject_id, nsubjects, nreps, lo_freq, hi_freq, win_size, step, exclude, score_list = ['f1'], \
                          nsets_training = 10,verbose = 0, epochs = 40, batch_size = 2, mv = False, permute = False):
    """
    Test naive cross-subject generalization of an RNN model
    Train an RNN model on data from one subject; test on all other subjects
    """


    subject_folder = os.path.join(data_folder,'%02d'%(src_subject_id))
    print('=======================')
    print(subject_folder)

    # Process data and get features 
    #get features across segments and corresponding info
    feature_matrix_src, target_labels_src, window_tstamps_src, \
    block_labels_src, series_labels_src = get_subject_data_for_classification(subject_folder, lo_freq, hi_freq, \
                                                                win_size, step)
    target_labels_src_orig = target_labels_src.copy()#keep originals before permuting
    train_idxs = np.arange(target_labels_src.size)

    np.random.seed(1)#for reproducibility

    results_df = []#initialize empty array for dataframes
    n_scores = len(score_list)
    train_scores_all = np.empty((nreps,n_scores))

    for rep in range(nreps):
        if permute:
            #permute while ignoring excluded blocks
            target_labels_src = permute_class_within_sub(target_labels_src_orig, block_labels_src, np.ones((target_labels_src.size,)), exclude)

        print('Subject %d|Rep %d'%(src_subject_id, rep+1))
        train_scores, trained_model, scaler, train_history = get_trained_rnn_model(feature_matrix_src, target_labels_src, train_idxs, block_labels_src,
                                                                                        nsets_training,exclude,\
                                                                                score_list = score_list,\
                                                                        verbose = verbose, epochs = epochs, batch_size = batch_size,\
                                                                        mv = mv)
        train_scores_all[rep,:] = train_scores

        # test on all other subjects
        # initialize empty lists
        test_scores_all = np.empty((0,0))
        targ_subject_list = []
        for targ_subject_id in range(1,nsubjects+1):
            if targ_subject_id != src_subject_id:

                subject_folder = os.path.join(data_folder,'%02d'%(targ_subject_id))
                print('Target Subject :%s'%(subject_folder))

                # Process data and get features 
                #get features across segments and corresponding info
                feature_matrix_targ, target_labels_targ, window_tstamps_targ, \
                block_labels_targ, series_labels_targ = get_subject_data_for_classification(subject_folder, lo_freq, hi_freq, \
                                                                        win_size, step)
                test_idxs = np.arange(target_labels_targ.size)

                test_scores = evaluate_trained_rnn(feature_matrix_targ, target_labels_targ, test_idxs, exclude, trained_model,\
                                                        score_list, scaler, mv = mv)
                #append to list
                test_scores_all = np.vstack((test_scores_all, test_scores)) if test_scores_all.size else test_scores
                targ_subject_list.append(targ_subject_id)

            #put testing results in dataframe
        data_dict = {'Type':['Test' for x in range(nsubjects-1)],\
                        'Rep':[rep+1 for x in range(nsubjects-1)],\
                        'Test_Subject':targ_subject_list}
        for sidx in range(n_scores):
            data_dict['%s_score'%(score_list[sidx])] = test_scores_all[:,sidx]
        results_df.append(pd.DataFrame(data_dict))

    # #put training results in dataframe
    data_dict = {'Type':['Train' for x in range(nreps)],\
                    'Rep':[x+1 for x in range(nreps)],\
                    'Test_Subject':[src_subject_id for x in range(nreps)]}
    for sidx in range(n_scores):
        data_dict['%s_score'%(score_list[sidx])] = train_scores_all[:,sidx]
    results_df.append(pd.DataFrame(data_dict))


    results_df = pd.concat(results_df, axis = 0).reset_index(drop = True)

    return results_df


In [11]:
results_df.groupby(['Type']).mean()

,Rep,Test_Subject,f1_score,accuracy_score
Type,,,,
Test,1.5,3.5,0.643046,0.676012
Train,1.5,1.0,0.997663,0.997664


In [12]:
results_df

,Type,Rep,Test_Subject,f1_score,accuracy_score
0,Test,1,2,0.274271,0.334928
1,Test,1,3,0.770124,0.784211
2,Test,1,4,0.717256,0.779621
3,Test,1,5,0.778579,0.801527
4,Test,2,2,0.288828,0.342105
5,Test,2,3,0.811905,0.831579
6,Test,2,4,0.791267,0.796209
7,Test,2,5,0.712141,0.737913
8,Train,1,1,0.997663,0.997664
9,Train,2,1,0.997663,0.997664


In [29]:
train_scores_all

array([[0.96426355, 0.96425234],
       [0.96116881, 0.96121495]])

In [21]:
nsubjects = 5

Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/02
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/03
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/04
Evaluate Model
Target Subject :/content/drive/MyDrive/EMG_gestures/EMG_data/05
Evaluate Model


In [26]:
test_scores_all

array([[0.27506061, 0.3277512 ],
       [0.75139437, 0.77368421],
       [0.75329157, 0.77725118],
       [0.59988813, 0.6437659 ]])